In [ ]:
import csv
from networksdb import NetworksDB
import pandas as pd
import datetime

In [ ]:
# OPTION 1 : import the key api named 'networksdb' from the keyring manager
# below the python cli (to execute once) create the api key : 
# >>> import keyring
# >>> import readline
# >>> keyring.set_password(service_name="networksdb",username="None",password="xxx")
# >>> readline.clear_history()
import keyring
keyring.get_keyring()
netdbio_apikey = keyring.get_credential("networksdb", "None")
netdbio_api = NetworksDB(netdbio_apikey.password)

In [ ]:
# OPTION 2 : import the key api named from the environment variable 
import os
netdbio_apikey = os.environ['NETDBIO_API_KEY']
netdbio_api = NetworksDB(netdbio_apikey)

In [ ]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

org_list = pd.read_csv(filename)
print(org_list)

In [ ]:
def save_ipranges_to_csv(org_name):
    
    org_data = netdbio_api.org_search(org_name)
    org_net_list = []
    
    for i in range(0,len(org_data.results)-1) :

        net_id = org_data.results[i].id
        ipranges = netdbio_api.org_networks(net_id)

        for ipr in ipranges.results:

            if ipr.cidr != "N/A":            
                ipr_sum = [ipr.netname, ipr.description, ipr.cidr]
                org_net_list.append(ipr_sum)

    pd.set_option('display.max_colwidth', None)            
    df = pd.DataFrame(org_net_list, columns=['net_name', 'desc', 'IP'])
    df.set_index('net_name', inplace=True)
    current_date = datetime.datetime.now()
    cdate = str(current_date.year) + f'{current_date.month:02d}' + f'{current_date.day:02d}'
    input_file = "NetDBio_IPranges_" + org_name + "_" + cdate
    df.to_csv(str(input_file + '.csv'))

In [ ]:
for index, org in org_list.iteritems():
    print(org)
    save_ipranges_to_csv(org)